## Question Type 성능 분석

### RULE MainQ Question Type별 성능 분석

1. MainQ와 MainQ Type의 매칭
2. 관찰된 성능과 ReClor 페이퍼의 관찰 결과를 비교하기 
    - (Type별 성능이 페이퍼에 리포트된 것과 비슷한 경향을 보이는지를 통해 관찰의 유효성을 검증)

실험 결과는 Mixtral 8x7B 8bit 결과를 사용

In [39]:
# result_file = 'Result/Mixtral_8bit_template:mixtral/Mixtral_template_RULE_mainq_Mixtral_8bit.jsonl'
# map_file = 'Result/RULE_mainq_map.jsonl'
# type_name_file = '../Data/ReClor/question_type_names.json'

result_file = 'Result/Mixtral_8bit_template:mixtral/Mixtral_template_RULE_subq_all_Mixtral_8bit.jsonl'
map_file = 'Result/RULE_subq_map.jsonl'
type_name_file = '../Data/ReClor/question_type_names.json'

import json
import pandas as pd 

result_data = pd.read_json(result_file, lines=True)
map_data = pd.read_json(map_file, lines=True)
type_name_data = pd.read_json(type_name_file)

# result_data에 response type 추출
result_data['response_code'] = map_data['response_code']
result_data['question_type'] = result_data['response_code'].apply(lambda x: type_name_data[0][x])

In [40]:
# result_data 정답 추출 (inference -> prediction)
result = result_data

def extract_first_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()]))-1

def correctness_matching(subq_all: pd.DataFrame, subq_id: str):
    if subq_id in subq_all['id_string'].values:
        return subq_all[subq_all['id_string']==subq_id]['main_option_correctness'].values[0]
    else: 
        return None

def extract_mainq_id_string(subq_id: str):
    return subq_id.split('_sub')[0]


result['prediction'] = result['inference'].apply(extract_first_num_in_string)
correct = 0
for i in range(len(result)):
    if result.label[i] == result.prediction[i]:
        correct += 1
print(f'result, accuracy: {correct/len(result)}')

result, accuracy: 0.6623376623376623


In [43]:
# result_selctive는 result에서 main_option_correctness가 True인 것만 추출
result_selective = result[result['main_option_correctness']==True]
result = result_selective

types_list = []
types_num_list = []
accuracy_list = []
num_list = []
for i in range(len(type_name_data)):
    # result에서 response_code가 i인 것만 추출
    result_i = result[result['response_code']==i]
    
    correct = 0
    for j in range(len(result_i)):
        if result_i.label.values[j] == result_i.prediction.values[j]:
            correct += 1
    
    if len(result_i) != 0:
        print(f'{type_name_data[0][i]}({i})\taccuracy: {correct/len(result_i)} ({len(result_i)})')
        types_list.append(type_name_data[0][i])
        types_num_list.append(i)
        accuracy_list.append(correct/len(result_i))
        num_list.append(len(result_i))
    
# df에 type_name_data[0][i], accuracy, len(result_i) 저장
    
df = pd.DataFrame()
df['type_name'] = types_list
df['type_num'] = types_num_list
df['accuracy'] = accuracy_list
df['num'] = num_list

df

,type_name,type_num,accuracy,num
